In [55]:
import os
run = "v3part"

In [56]:
rez = []
for it in os.listdir(run):
    work_dir = os.path.join(run, it)
    with open(os.path.join(work_dir, "cross_val_rez.txt"), "r") as f:
        lines = f.readlines()
    rez.append((float(lines[0].strip()), it))

In [57]:
rez.sort()

In [58]:
rez = rez[::-1]

In [59]:
wds = [x[1] for x in rez]#[:3]

In [60]:
wds

['1', '3', '2', '0', '4']

In [61]:
import pandas as pd
trains = []
tests = []
for wd in wds:
    work_dir = os.path.join(run, wd)
    train = pd.read_csv(os.path.join(work_dir, "train.csv")).drop(columns=["Unnamed: 0"])
    test = pd.read_csv(os.path.join(work_dir, "test.csv")).drop(columns=["Unnamed: 0"])
    order = train.columns.values
    trains.append(train.values)
    tests.append(test.values)
    

In [62]:
import numpy as np
train_mean = np.stack(trains).mean(axis=0)
test_mean = np.stack(tests).mean(axis=0)

In [63]:
train_df = pd.DataFrame(train_mean, columns=order)
test_df = pd.DataFrame(test_mean, columns=order)

In [64]:
train_df.to_csv("consolidated_train_" + run + '.csv')
test_df.to_csv("consolidated_test_" + run + '.csv')

In [65]:
from sklearn.metrics import roc_auc_score
def weighted_roc_auc(y_true, y_pred, labels, weights_dict):
    unnorm_weights = np.array([weights_dict[label] for label in labels])
    weights = unnorm_weights / unnorm_weights.sum()
    classes_roc_auc = roc_auc_score(y_true, y_pred, #labels=labels,
                                    multi_class="ovr", average=None)
    return sum(weights * classes_roc_auc)

cluster_weights = pd.read_excel("./cluster_weights.xlsx").set_index("cluster")
weights_dict = cluster_weights["unnorm_weight"].to_dict()


train = pd.read_parquet('train_data.pqt', engine='pyarrow')
y_train = train[train.date=="month_3"].copy().reset_index()["end_cluster"].values
label2pos = {x:e for e, x in enumerate(order)}
y_train_id = [label2pos[x] for x in y_train]

In [66]:
weighted_roc_auc(y_train_id, train_mean, order, weights_dict)

0.9127856109158521

In [54]:
train.start_cluster.value_counts()

{α}          375947
{}            78803
{α, η}        43824
{α, γ}        32790
{other}       31813
{α, β}         9869
{α, δ}         8537
{α, ε}         5843
{α, θ}         4968
{α, ψ}         3000
{α, μ}         1341
{α, ε, η}      1207
{α, ε, θ}       851
{α, λ}          739
{α, ε, ψ}       374
{λ}              69
{α, π}           25
Name: start_cluster, dtype: int64

In [26]:
train[train["start_cluster"] == '{α, μ}'].end_cluster.value_counts()

{α, μ}       682
{α}          389
{other}      199
{}            38
{α, η}        14
{α, γ}         7
{α, β}         5
{α, θ}         3
{α, ε, η}      2
{α, δ}         1
{α, ε}         1
Name: end_cluster, dtype: int64